In [1]:
import cv2
import numpy as np
import pandas as pd
import os

from sklearn.cluster import KMeans

from imutils import face_utils
import dlib

In [2]:
n_colors = 4

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
data_root = './images/dataset'
seasons = ['spring', 'summer', 'fall', 'winter']

In [4]:
def bgr2cmyk(bgr):
    res_cmyk = []
    bgr = bgr / 255.0
    
    for i in range(len(bgr)):
        b, g, r = bgr[i]
        k = 1 - max(r, g, b)
        c = (1 - r - k) / (1 - k) * 255
        m = (1 - g - k) / (1 - k) * 255
        y = (1 - b - k) / (1 - k) * 255

        tmp = np.array([c, m, y, k])
        res_cmyk.append(tmp)
        
    return np.array(res_cmyk)

In [5]:
lips = None
left_cheek = None
right_cheek = None
right_eye = None
left_eye = None
nose = None

def extract_face_part(img, face_part_points):

    (x, y, w, h) = cv2.boundingRect(face_part_points)
    crop = img[y:y+h, x:x+w]
    
    # https://www.researchgate.net/publication/262371199_Explicit_image_detection_using_YCbCr_space_color_model_as_skin_detection
    # filter skin only (YCbCr)
    crop = cv2.cvtColor(crop, cv2.COLOR_BGR2YCrCb)
    mask = cv2.inRange(crop, np.array([0, 133, 77]), np.array([255, 173, 127]))
    crop = cv2.bitwise_and(crop, crop, mask=mask)
    crop = cv2.cvtColor(crop, cv2.COLOR_YCrCb2BGR)

    crop = crop[~np.all(crop == [0, 135, 0], axis=-1)]
    crop = crop.reshape(((1, crop.shape[0], 3)))
    
    return crop

def detect_face_part(img):
    face_parts = [[] for _ in range(len(face_utils.FACIAL_LANDMARKS_IDXS))]

    faces = detector(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 1)
    
    if len(faces) == 0:
        return 0

    rect = faces[0]

    shape = predictor(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), rect)
    shape = face_utils.shape_to_np(shape)

    for idx, (_, (i, j)) in enumerate(face_utils.FACIAL_LANDMARKS_IDXS.items()):
        if idx not in [1, 3]:
            face_parts[idx] = shape[i:j]

    lips = extract_face_part(img, np.concatenate((shape[48:60], shape[60:68])))
    left_cheek = extract_face_part(img, np.concatenate((shape[29:33], shape[4:9])))
    right_cheek = extract_face_part(img, np.concatenate((shape[29:33], shape[10:15])))
    right_eye = extract_face_part(img, shape[36:42])
    left_eye = extract_face_part(img, shape[42:48])
    nose = extract_face_part(img, shape[27:36])

    return len(faces), lips, left_cheek, right_cheek, right_eye, left_eye, nose

def create_palette(img, image_path='image.jpg'):
        
    img = cv2.imread(image_path)
    
    if img is None:
        os.remove(image_path)
        return None
        
    yes_faces, lips, left_cheek, right_cheek, right_eye, left_eye, nose = detect_face_part(img)
    
    if not yes_faces:
        os.remove(image_path)
        return None
    
    stacked_images = np.hstack([right_eye, left_eye, lips, left_cheek, right_cheek, nose])
    stacked_images = stacked_images.reshape(-1, 3)
    
    if stacked_images.shape[0] == 0:
        os.remove(image_path)
        return None
        
    kmeans = KMeans(n_clusters=n_colors, n_init=10, random_state=42)
    kmeans.fit(stacked_images)

    cluster_centers = kmeans.cluster_centers_.astype(int)
    
    # save_palette(cluster_centers)
    
    return cluster_centers, lips, left_cheek, right_cheek, right_eye, left_eye, nose

In [6]:
def calculate_contrast(img):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])
    hist /= hist.sum()
    mean = np.mean(hist)
    variance = np.mean((hist - mean) ** 2)
    return variance

In [7]:
for season in seasons:
    this_df = pd.DataFrame(columns=['lip_avg_a', 'face_l_var', 'skin_avg_b', 'H', 'S', 'V', 'L', 'a', 'b', 'C', 'M', 'Y', 'K', 'label'])
    for file in os.listdir(os.path.join(data_root, season)):
        full_path = os.path.join(data_root, season, file)
        img = cv2.imread(full_path)
        if img is None:
            continue
        palette, lips, left_cheek, right_cheek, right_eye, left_eye, nose = create_palette(full_path)
        if palette is None:
            continue
        cmyk_palette = bgr2cmyk(palette)
        palette = np.array([palette], np.uint8)
        hsv_palette = cv2.cvtColor(palette, cv2.COLOR_BGR2HSV)
        lab_palette = cv2.cvtColor(palette, cv2.COLOR_BGR2LAB)
    
        mean_hsv = np.mean(hsv_palette, axis=1)[0]
        mean_lab = np.mean(lab_palette, axis=1)[0]
        mean_cmyk = np.mean(cmyk_palette, axis=0)
        
        skin = np.hstack([left_cheek, right_cheek, nose])
        skin = skin.reshape(-1, 3)
        kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
        kmeans.fit(skin)
        skin_centers = kmeans.cluster_centers_.astype(int)
        skin_centers = np.array([skin_centers], np.uint8)
        lab_palette = cv2.cvtColor(skin_centers, cv2.COLOR_BGR2LAB)
        mean_lab_skin = np.mean(lab_palette, axis=1)[0]
        
        kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
        lips = lips.reshape(-1, 3)
        kmeans.fit(lips)
        lips_centers = kmeans.cluster_centers_.astype(int)
        lips_centers = np.array([lips_centers], np.uint8)
        lab_palette = cv2.cvtColor(lips_centers, cv2.COLOR_BGR2LAB)
        mean_lab_lips = np.mean(lab_palette, axis=1)[0]
        
        h, w = img.shape[:2]
        face = detector(img)[0]
        face = img[max(0, face.top()):min(face.bottom(), h), max(0, face.left()):min(face.right(), w)]
        face_l_var = calculate_contrast(face)
        
        tmp = np.array([mean_lab_lips[1], face_l_var, mean_lab_skin[2]])
        row = np.concatenate((tmp, mean_hsv, mean_lab, mean_cmyk)).tolist()
        row.append(season)
        this_df.loc[len(this_df)] = row
    print(f'{season}: {len(this_df)} rows')
    this_df.to_csv(f'images/dataset/mean_{season}.csv', index=False)

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


In [ ]:
df = pd.concat([pd.read_csv(f'images/dataset/mean_{season}.csv') for season in seasons])
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv('images/dataset/mean_shuffled.csv', index=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [ ]:
corr = df.corr()
corr['label'].sort_values(ascending=False)